In [1]:
using Laplacians

objc[18981]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[18981]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[18981]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[18981]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [2]:
include("treeSolver.jl")
include("IO.jl")
include("generators.jl")
include("toHighPrecision.jl")
include("verbosePCG.jl")

pcgWorker (generic function with 1 method)

In [3]:
pwd()

"/Users/serbanstan/git/TreePCG/julia"

In [4]:
graphp = "../graphs/se_4/"

"../graphs/se_4/"

In [5]:
a = readFromFile(graphp * "graph.mtx");

In [6]:
tree = readFromFile(graphp * "tree.mtx");

In [7]:
x = readFromFile(graphp * "x.vec");

In [8]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [10]:
@time F = treeSolver(tree)
@time F(b);

 11.643632 seconds (2.50 M allocations: 171.090 MB, 0.16% gc time)
  0.438585 seconds (4.00 M allocations: 76.256 MB, 39.48% gc time)


In [13]:
# a run on standard Float64 data types
numIts = 100;
@time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=true);
norm(lap(a) * myx64 - b) / norm(b)
insert!(dbg, 1, "$(numIts) iterations")
insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
writeToFile(graphp * "_log_julia_default.txt", dbg)

Working on iteration 10 with a-norm error 1027.4932372948738
Working on iteration 20 with a-norm error 15.814999020407681
Working on iteration 30 with a-norm error 0.26826881729252894
Working on iteration 40 with a-norm error 0.0033043762455750108
Working on iteration 50 with a-norm error 1.7909890094811588e-5
Working on iteration 60 with a-norm error 6.3462218201432895e-6
Working on iteration 70 with a-norm error 6.3200121819824545e-6
Working on iteration 80 with a-norm error 6.32292162018948e-6
Working on iteration 90 with a-norm error 6.322943225213564e-6
Working on iteration 100 with a-norm error 6.322947011938211e-6
PCG stopped after: 100 iterations with relative error 6.3229487006264386e-6.
 23.226156 seconds (403.76 M allocations: 10.217 GB, 15.61% gc time)


In [14]:
numIts = 100
for hp in [64,128,1024]
    set_bigfloat_precision(hp)
    laHP = toHighPrecision(la; precision=hp)
    treeHP = toHighPrecision(tree; precision=hp)
    xHP = toHighPrecision(x; precision=hp);
    bHP = toHighPrecision(b; precision=hp);
    FHP = treeSolver(treeHP)
    
    @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_$(hp).txt", dbg)
end

Working on iteration 10 with a-norm error 1024.7007140598735
Working on iteration 20 with a-norm error 7.514398764542579
Working on iteration 30 with a-norm error 0.09706993497803132
Working on iteration 40 with a-norm error 0.00020053336044311055
Working on iteration 50 with a-norm error 9.84452822885789e-7
Working on iteration 60 with a-norm error 5.805083494030058e-8
Working on iteration 70 with a-norm error 5.322765998617783e-8
Working on iteration 80 with a-norm error 5.3203255865330416e-8
Working on iteration 90 with a-norm error 5.3202049465289506e-8
Working on iteration 100 with a-norm error 5.3202048035326195e-8
PCG stopped after: 100 iterations with relative error 5.3202047871259295e-8.
2328.058637 seconds (6.45 G allocations: 163.371 GB, 47.22% gc time)
Working on iteration 10 with a-norm error 685.338875738586
Working on iteration 20 with a-norm error 1.4734586105163687
Working on iteration 30 with a-norm error 0.0007141158477822725
Working on iteration 40 with a-norm error

In [ ]:
?pcgV

In [ ]:
typeof(laHP)